
    
    
## <H2 align="center"> <font color='white'> **MAF1731B**: Microstructure and Trading Systems </H2>

### <H3 align="center"> <font color='white'> **Laboratory 4**: Market Microstructure </H3>

 <H5 align="center"> <em> Professor: Villalobos Ramos Omar Antonio <em>

 <p align="center"> <img style=" margin: 30px 15px 15px 15px;" src="https://pngimage.net/wp-content/uploads/2018/06/logo-iteso-png-5.png" width="350" height="150" /> 

 <strong> <H5 align="left"> Student:
Esteban Márquez Delgado <strong>

 *Registry*:
if700637: https://github.com/EstebanMqz/MyST_LAB_4

 <center> <font color= #555555> <font size = 4> November 2022 </a> <font color= #555555> <font size = 4> </a></font>

<hr style="border:0.02in solid gray"> </hr>

<center> <font color= #555555> <font size = 6> Laboratory 4 </font> <br> <br> <font color= #555555> <font size = 5> Market Microstructure  </font>

<hr style="border:0.02in solid gray"> </hr>

### <font color= #555555> <font size = 4> Abstract </font>

This document was prepared as delivery for Laboratory 4 of Microstructure and Trading Systems - MAF1731B class during the Autumn 2022 course at ITESO for the Bachelor of Financial Engineering. In order to analyze the market microstructure in criptocurrencies markets containing the following 3 sections:

1. **Consume CCXT data**: Download Criptocurrencies Order Books from exchanges (kraken, ftx, currencycom, coinmate, etc).

2. **Microstructure Visualization**: Build Time Series using the Order Books obtained in the previous point from the following elements of the Market Microstructure for each asset and exchange (Levels, Bid Volume, Ask Volume, Total Volume, Mid Price, VWAP.)

3. **Microstructure Modeling**: Estimation of Roll Effective Spread using the lagged series at t-5.



<font color= lightblue> <font size = 4> 0. Libraries and Dependencies: </font>

<font color= gray>

**Note**: To run this notebook it is necessary to have the following libraries contained in the [requirements.txt](https://github.com/EstebanMqz/MyST_LAB_4/blob/main/requirements.txt) script of this project.

To install the libraries in the script run the following cell:

In [1]:
!pip install -r requirements.txt

ERROR: Invalid requirement: '"""' (from line 1 of requirements.txt)


If you prefer, manual installation can be done with the individual installation of the following libraries:

<font color= lightblue> <font size = 1> 
+ pandas >= 1.3.4
+ numpy >= 1.19.1
+ jupyter >= 1.0.0
+ chart_studio >= 1.1
+ plotly >= 4.14
+ logging >= 3.11
+ datetime >= 4.7
+ pandas_datareader >= 0.10.0
+ tk >= 0.10
+ random >= 3.11
+ plotly >= 5.60
+ yahoofinancials >= 1.60
+ scipy >= 1.7.3
+ regex >= 2022.3.15
+ matplotlib >= 3.5.1
+ fire >= 0.4.0
+ ccxt >= 2.1.33
+ time >= 3.11 


### <font color= lightblue> <font size = 4> 0.1 Introduction: </font>
In this laboratory we will use information from the Cryptocurrency Order Book of various international exchanges, with the primary objective of visualizing and modeling the microstructure of the market through the behavior of the Order Book and its characteristics such as; Matching Engine, Levels, Ask-Bid Volumes, etc.
<font color= gray>




##### *General help:*
<font color= lightgreen> **Note**: Progress checks in sections highlighted. <font color= white>

In order, to retrieve specific help from the scripts tree in the project execute the help function on the script or in specific modules from the following:
+ [data.py](https://github.com/EstebanMqz/MyST_LAB_4/blob/main/data.py)
+ [functions.py](https://github.com/EstebanMqz/MyST_LAB_4/blob/main/functions.py)
+ [visualizations.py](https://github.com/EstebanMqz/MyST_LAB_4/blob/main/visualizations.py)

In [2]:
## Libraries
import pandas as pd # Analysis and data management
import pandas_datareader as pdr
import pandas_datareader.data as web #Data extraction from the web
import numpy as np
from datetime import datetime, timedelta, date
from tkinter.ttk import Style
from tkinter import Y
import random as rand
import plotly.graph_objects as go #Plotly
import plotly.express as px
import warnings
from yahoofinancials import YahooFinancials # Yfinance 
import warnings
import yfinance as yf
import ccxt #Criptocurrencies
import logging as log #Exchanges 
from scipy.optimize import minimize # Optimization 
from collections import Counter #Counter
import time
import re
import io 
import glob
import os
#.Py archives in repo
import functions as fn 
import visualizations as vs
import data as dt
from os import path
import fire
%matplotlib inline

<font color= orange> <font size = 5> 1. Consume CCXT data: </font>

In order to know the exchanges avaliable in CCTX module in Python for Criptocurrencies data download, the following getattr function *find_exchanges* is executed:

In [3]:
help(fn.find_exchanges)

Help on function find_exchanges in module functions:

find_exchanges(features=None, is_authenticated=False)
    Function that returns avaliable cryptocurrencies exchanges in Python for CCTX module.
    
        Parameters:
        ----------
        features: None 
        is_authenticated: None 
    
        Returns:
        -------
        exchange_names: Array of authenticated exchanges avaliable in CCTX.



From all the 120 exchanges avaliable we pick **binance**, **ftx** and **bytetrade** because of their fast fetching.

In [4]:
#Available exchanges in CCTX
fn.find_exchanges()[:] #len 120

['aax',
 'alpaca',
 'ascendex',
 'bequant',
 'bibox',
 'bigone',
 'binance',
 'binancecoinm',
 'binanceus',
 'binanceusdm',
 'bit2c',
 'bitbank',
 'bitbay',
 'bitbns',
 'bitcoincom',
 'bitfinex',
 'bitfinex2',
 'bitflyer',
 'bitforex',
 'bitget',
 'bithumb',
 'bitmart',
 'bitmex',
 'bitopro',
 'bitpanda',
 'bitrue',
 'bitso',
 'bitstamp',
 'bitstamp1',
 'bittrex',
 'bitvavo',
 'bkex',
 'bl3p',
 'blockchaincom',
 'btcalpha',
 'btcbox',
 'btcex',
 'btcmarkets',
 'btctradeua',
 'btcturk',
 'buda',
 'bw',
 'bybit',
 'bytetrade',
 'cex',
 'coinbase',
 'coinbaseprime',
 'coinbasepro',
 'coincheck',
 'coinex',
 'coinfalcon',
 'coinmate',
 'coinone',
 'coinspot',
 'crex24',
 'cryptocom',
 'currencycom',
 'delta',
 'deribit',
 'digifinex',
 'exmo',
 'flowbtc',
 'fmfwio',
 'ftx',
 'ftxus',
 'gate',
 'gateio',
 'gemini',
 'hitbtc',
 'hitbtc3',
 'hollaex',
 'huobi',
 'huobijp',
 'huobipro',
 'idex',
 'independentreserve',
 'indodax',
 'itbit',
 'kraken',
 'kucoin',
 'kucoinfutures',
 'kuna',
 'lat

Now that we have selected the exchanges, we pick random criptocurrencies and verify that they are all avaliable in their platform.

In [5]:
exchanges= ['binance','ftx', 'bytetrade'] #Selected exchanges from list (can't modify because of fn and vs .py)
criptos = ['BTC/USDT','ETH/USDT','XRP/USDT'] #Criptos OBs (modify tickers here)

<font color= lightblue> <font size = 4> Order Book Levels & OHLCV for 3 exchanges and Criptos: </font>

Now we proceed to download Single Order Books from exchanges for criptocurrencies.

In [6]:
help(vs.cctx_download)

Help on function cctx_download in module visualizations:

cctx_download(lvls, cripto, exchange)
    Function that returns prices and quantities of given levels (as integer) Bids & Asks in an Orderbook
    for the specified criptocurrency ('BTC/USDT','ETH/USDT','XRP/USDT', or others) and
    from the following exchanges: binance, ftx or ascendex (as string).
    
        Parameters:
        ----------
        lvls: Levels of bids/asks in the Order Book (int).
        cripto: Criptocurrency downloadable symbol (str).
        exchanges: Criptocurrency downloadable Exchange (str).
    
        Returns:
        -------
        levels_ob_bid: Prices and Quantities of Bids (pos [0]) and Asks (pos [1]) as dataframes.



We get n=25 levels for each OB, and proceed to download criptos and exchanges from their lists [0:2] (9).

We will start with binance and BTC/USDT.

In [7]:
params= 25, criptos[0], exchanges[0]
params

(25, 'BTC/USDT', 'binance')

+ <font color= lightgray> <font size = 2> *Bids*: </font>

In [8]:
ob_lvls=vs.cctx_download(params[0],params[1], params[2])
ob_lvls[0].head()

,price,quantity
Bid_Lvl,,
0,17395.07,0.04000
1,17395.06,0.00672
2,17394.68,0.05111
3,17394.67,0.05000
4,17394.32,0.01200


+ <font color= lightgray> <font size = 2> *Asks*: </font>

In [9]:
ob_lvls[1].head()

,price,quantity
Ask_Lvl,,
0,17396.79,0.14378
1,17396.80,0.00589
2,17397.18,0.06250
3,17397.19,0.00600
4,17397.50,0.00663


In [10]:
ob_lvls[2]

,exchange,timestamp,open,high,low,close,volume
OHLCV,,,,,,,
0,binance,2022-11-10T18:14:16.194Z,17205.47,17269.0,17190.04,17260.11,612.15596


We plot an **Order Book** for desired tickers and exchanges the visualization *OBLvls*.

In [11]:
help(vs.OBLvls_hist)

Help on function OBLvls_hist in module visualizations:

OBLvls_hist(lvls, cripto, exchange)
    Function that plots an horizontal histogram in plotly for CriptoCurrencies OB/
    
        Parameters
        ----------
        lvls: Levels of bids/asks in the Order Book (int).
        cripto: Criptocurrency downloadable symbol (str).
        exchanges: Criptocurrency downloadable Exchange (str).
    
        x: Quantity (col) of the given cripto (str) for given lvls (int) in Order Book.
        y: Prices (col) of the given cripto (str) for given lvls (int) in Order Book.
    
        Returns
        -------
        Histogram of Order Book prices and quantities for n given lvls.



In [12]:
vs.OBLvls_hist(params[0],params[1], params[2])

We will proceed for binance and ETH/USDT.

In [13]:
params= 25, criptos[1], exchanges[0]
params

(25, 'ETH/USDT', 'binance')

In [14]:
ob_lvls=vs.cctx_download(params[0],params[1], params[2])
ob_lvls[0].head()

,price,quantity
Bid_Lvl,,
0,1275.97,3.7793
1,1275.87,0.0500
2,1275.85,0.0156
3,1275.70,0.3817
4,1275.44,1.0732


In [15]:
ob_lvls[1].head()

,price,quantity
Ask_Lvl,,
0,1275.98,2.3399
1,1276.10,0.5000
2,1276.11,0.5581
3,1276.13,0.0930
4,1276.26,0.0432


In [16]:
ob_lvls[2]

,exchange,timestamp,open,high,low,close,volume
OHLCV,,,,,,,
0,binance,2022-11-10T18:14:20.530Z,1259.74,1267.41,1258.44,1264.93,1680.795


In [17]:
vs.OBLvls_hist(params[0],params[1], params[2])

We continue with the exchange binance and XRP/USDT.

In [18]:
params= 25, criptos[2], exchanges[0]
params

(25, 'XRP/USDT', 'binance')

In [19]:
ob_lvls=vs.cctx_download(params[0],params[1], params[2])
ob_lvls[0].head()

,price,quantity
Bid_Lvl,,
0,0.3802,8159.0
1,0.3801,11296.0
2,0.3800,9849.0
3,0.3799,27181.0
4,0.3798,125390.0


In [20]:
ob_lvls[1].head()

,price,quantity
Ask_Lvl,,
0,0.3803,3141.0
1,0.3804,3841.0
2,0.3805,15137.0
3,0.3806,27752.0
4,0.3807,38810.0


In [21]:
ob_lvls[2]

,exchange,timestamp,open,high,low,close,volume
OHLCV,,,,,,,
0,binance,2022-11-10T18:14:24.563Z,0.377,0.3785,0.3768,0.3784,432692.0


In [22]:
vs.OBLvls_hist(params[0],params[1], params[2])

We could retrieve criptocurrencies with any of the 3 exchanges declared in our list *exchanges* but **to avoid being repetitive we will make one more random example** and we will conclude with their dataframes in progress check. For section section **2.** (*Microstructure Visualization*) a dataframe with different columns and plots that allow us to compare differences between exchanges will be created.

In [71]:
params= 25, criptos[rand.randint(0,2)], exchanges[rand.randint(0,2)]
params

(25, 'XRP/USDT', 'bytetrade')

In [72]:
ob_lvls=vs.cctx_download(params[0],params[1], params[2])

As it can be seen, liquidity is a decisive factor between exchanges. This is because there are many levels in OBs for most criptocurrencies that have big volumes due to their inherent volatility and this is also related to market makers participation.

In [73]:
vs.OBLvls_hist(params[0],params[1], params[2])

In [74]:
v_params0= 25, criptos[0], exchanges[0]
v_params1= 25, criptos[0], exchanges[1]
v_params2= 25, criptos[0], exchanges[2]
v_params0, v_params1, v_params2

((25, 'BTC/USDT', 'binance'),
 (25, 'BTC/USDT', 'ftx'),
 (25, 'BTC/USDT', 'bytetrade'))

<strong> 
<font color= lightgreen> <font size = 3> Progress check 1 </font>

<font color= orange> <font size = 1> n = data fetch per exchange </font>

In [84]:
n=20 #Iterations 

In [85]:
help(vs.verif_ex1)

Help on function verif_ex1 in module visualizations:

verif_ex1(lvls, cripto, exchange, n)
    Function that returns Verif 1 for i levels (as integer) n times 
    from specified criptocurrency ('BTC/USDT','ETH/USDT','XRP/USDT', or others) and the 
    following exchanges: binance, ftx or bytetrade (as string).
    
    Parameters:
    ----------
    lvls: Levels of bids/asks in the Order Book (int).
    cripto: Criptocurrency downloadable symbol (str).
    exchanges: Criptocurrency downloadable Exchange (str).
    n: Data retrieval per exchange (int).
    
    Returns:
    -------
    df1 = Section 1 Verification.
    df2 = OHLCV.



In [87]:
verif_0_B = vs.verif_ex1(v_params0[0],v_params0[1], v_params0[2], n)
verif_1_B = vs.verif_ex1(v_params1[0],v_params1[1], v_params1[2], n)
verif_2_B = vs.verif_ex1(v_params2[0],v_params2[1], v_params2[2], n)

Some differences between criptocurrencies within exchanges can be appreciated. Now we can realize which exchange from the sample can provide us with the lowest spread and best liquidity, and it is binance in this case. 

Clearly, prices are the most important factor for investments and that's why Mid-Price is going to be analyzed in the plots of section *2° (Microstructure Visualization)*. 

<font color= lightblue> <font size = 2> Verification BTC </font>

As it can be appreciated lower close prices of exchanges can be associated with bigger bid volumes mainly.

In [88]:
verif_df1 = fn.concatenate(verif_0_B[0], verif_1_B[0], verif_2_B[0], n)
verif_df1.head(10)

,exchange,timestamp,ask,bid,ask_volume,bid_volume,spread,close_price
0,binance,2022-11-10T18:32:22.278,17273.08,17271.25,2.23454,1.09826,1.83,17273.25
1,binance,2022-11-10T18:32:24.839,17280.22,17277.43,0.62428,1.29755,2.79,17279.33
2,binance,2022-11-10T18:32:27.867,17277.59,17276.65,0.92037,0.65863,0.94,17276.85
3,binance,2022-11-10T18:32:30.297,17274.99,17274.06,2.18523,1.11297,0.93,17274.99
4,binance,2022-11-10T18:32:32.721,17272.27,17270.96,1.43612,0.73102,1.31,17272.00
5,binance,2022-11-10T18:32:35.140,17273.99,17271.95,1.14332,2.76171,2.04,17272.10
6,binance,2022-11-10T18:32:37.636,17279.89,17278.61,0.65454,0.50760,1.28,17279.84
7,binance,2022-11-10T18:32:40.288,17279.28,17278.12,2.78923,0.80427,1.16,17278.12
8,binance,2022-11-10T18:32:42.753,17279.89,17279.30,0.66569,1.53481,0.59,17279.90
9,binance,2022-11-10T18:32:45.116,17275.47,17273.85,0.75127,0.60324,1.62,17279.70


In [89]:
verif_df1.tail(10)

,exchange,timestamp,ask,bid,ask_volume,bid_volume,spread,close_price
50,bytetrade,2022-11-10T18:34:48.048,17393.0,17136.3,2.71083,2.836563,256.7,17272.7
51,bytetrade,2022-11-10T18:34:50.729,17393.0,17136.3,2.71083,2.836563,256.7,17264.8
52,bytetrade,2022-11-10T18:34:53.407,17393.0,17136.3,2.71083,2.836563,256.7,17271.3
53,bytetrade,2022-11-10T18:34:56.918,17393.0,17136.3,2.71083,2.836563,256.7,17255.0
54,bytetrade,2022-11-10T18:34:59.461,17393.0,17136.3,2.71083,2.836563,256.7,17258.4
55,bytetrade,2022-11-10T18:35:02.243,17393.0,17136.3,2.71083,2.836563,256.7,17261.4
56,bytetrade,2022-11-10T18:35:04.938,17393.0,17136.3,2.71083,2.836563,256.7,17263.9
57,bytetrade,2022-11-10T18:35:07.738,17393.0,17136.3,2.71083,2.836563,256.7,17258.2
58,bytetrade,2022-11-10T18:35:10.427,17393.0,17136.3,2.71083,2.836563,256.7,17269.3
59,bytetrade,2022-11-10T18:35:12.769,17393.0,17136.3,2.71083,2.836563,256.7,17264.7


<font color= lightblue> <font size = 2> OHLCV BTC </font>

For each timeframe a candle is formed and it illustrates how much Volatility every timestamp shows within exchanges. This is important to know for algorithmic trading, not to mention the fact that stock exchanges do not provide candle data in *ms* for many reasons.

In [90]:
verif_df1_OH = fn.concatenate(verif_0_B[1], verif_1_B[1], verif_2_B[1], n)
verif_df1_OH.head(10)

,exchange,timestamp,open,high,low,close,volume
0,binance,2022-11-10T18:32:22.278,17257.89,17283.23,17251.9,17273.25,69.41647
1,binance,2022-11-10T18:32:24.839,17257.89,17283.23,17251.9,17279.33,72.87468
2,binance,2022-11-10T18:32:27.867,17257.89,17285.14,17251.9,17276.85,88.42610
3,binance,2022-11-10T18:32:30.297,17257.89,17285.14,17251.9,17274.99,97.14622
4,binance,2022-11-10T18:32:32.721,17257.89,17285.14,17251.9,17272.00,115.89142
5,binance,2022-11-10T18:32:35.140,17257.89,17285.14,17251.9,17272.10,122.73878
6,binance,2022-11-10T18:32:37.636,17257.89,17285.14,17251.9,17279.84,130.06146
7,binance,2022-11-10T18:32:40.288,17257.89,17285.14,17251.9,17278.12,134.42327
8,binance,2022-11-10T18:32:42.753,17257.89,17285.32,17251.9,17279.90,141.43061
9,binance,2022-11-10T18:32:45.116,17257.89,17285.32,17251.9,17279.70,157.82446


In [91]:
verif_df1_OH.tail(10)

,exchange,timestamp,open,high,low,close,volume
50,bytetrade,2022-11-10T18:34:48.048,17261.4,17280.7,17247.4,17272.7,0.05994
51,bytetrade,2022-11-10T18:34:50.729,17261.4,17280.7,17247.4,17264.8,0.06755
52,bytetrade,2022-11-10T18:34:53.407,17261.4,17280.7,17247.4,17271.3,0.06944
53,bytetrade,2022-11-10T18:34:56.918,17261.4,17280.7,17247.4,17255.0,0.07494
54,bytetrade,2022-11-10T18:34:59.461,17261.4,17280.7,17247.4,17258.4,0.08099
55,bytetrade,2022-11-10T18:35:02.243,17261.4,17261.4,17261.4,17261.4,0.00973
56,bytetrade,2022-11-10T18:35:04.938,17261.4,17263.9,17261.4,17263.9,0.01852
57,bytetrade,2022-11-10T18:35:07.738,17261.4,17263.9,17258.2,17258.2,0.02176
58,bytetrade,2022-11-10T18:35:10.427,17261.4,17269.3,17258.2,17269.3,0.02521
59,bytetrade,2022-11-10T18:35:12.769,17261.4,17269.3,17258.2,17264.7,0.02905


<font color= lightblue> <font size = 2> Verification ETH </font>

In [92]:
v_params0= 25, criptos[1], exchanges[0]
v_params1= 25, criptos[1], exchanges[1]
v_params2= 25, criptos[1], exchanges[2]
v_params0, v_params1, v_params2

((25, 'ETH/USDT', 'binance'),
 (25, 'ETH/USDT', 'ftx'),
 (25, 'ETH/USDT', 'bytetrade'))

In [93]:
verif_0_E = vs.verif_ex1(v_params0[0],v_params0[1], v_params0[2], n)
verif_1_E = vs.verif_ex1(v_params1[0],v_params1[1], v_params1[2], n)
verif_2_E = vs.verif_ex1(v_params2[0],v_params2[1], v_params2[2], n)

In [94]:
verif_df2 = fn.concatenate(verif_0_E[0], verif_1_E[0], verif_2_E[0], n)
verif_df2.head(10)

,exchange,timestamp,ask,bid,ask_volume,bid_volume,spread,close_price
0,binance,2022-11-10T18:35:16.488,1260.85,1260.84,68.8534,28.4608,0.01,1260.82
1,binance,2022-11-10T18:35:19.352,1260.99,1260.98,45.5307,87.7374,0.01,1260.81
2,binance,2022-11-10T18:35:21.798,1260.71,1260.67,62.1715,32.9065,0.04,1260.70
3,binance,2022-11-10T18:35:25.257,1260.78,1260.77,327.6291,36.5636,0.01,1260.77
4,binance,2022-11-10T18:35:27.654,1260.91,1260.90,340.2359,28.8660,0.01,1260.91
5,binance,2022-11-10T18:35:30.056,1260.87,1260.86,336.8576,30.9022,0.01,1260.86
6,binance,2022-11-10T18:35:33.453,1260.68,1260.67,332.7970,36.0167,0.01,1260.69
7,binance,2022-11-10T18:35:36.452,1260.83,1260.82,330.8043,49.8522,0.01,1260.83
8,binance,2022-11-10T18:35:39.016,1260.68,1260.67,354.6795,37.5509,0.01,1260.68
9,binance,2022-11-10T18:35:41.404,1260.53,1260.52,338.3205,31.9294,0.01,1260.55


In [95]:
verif_df2.tail(10)

,exchange,timestamp,ask,bid,ask_volume,bid_volume,spread,close_price
50,bytetrade,2022-11-10T18:37:51.755,1268.69,1251.24,16.5507,61.4832,17.45,1259.73
51,bytetrade,2022-11-10T18:37:54.640,1268.69,1251.24,16.4235,161.4021,17.45,1259.73
52,bytetrade,2022-11-10T18:37:58.846,1268.69,1251.24,19.5131,61.4834,17.45,1259.40
53,bytetrade,2022-11-10T18:38:01.451,1268.69,1251.24,19.5131,61.4834,17.45,1260.60
54,bytetrade,2022-11-10T18:38:04.970,1268.69,1251.24,18.9853,61.4549,17.45,1260.60
55,bytetrade,2022-11-10T18:38:07.708,1268.69,1251.24,18.9853,61.4549,17.45,1260.24
56,bytetrade,2022-11-10T18:38:10.964,1268.69,1251.24,18.9853,61.4549,17.45,1260.24
57,bytetrade,2022-11-10T18:38:13.930,1268.69,1251.24,18.7800,61.4549,17.45,1259.13
58,bytetrade,2022-11-10T18:38:16.717,1268.69,1251.24,18.7800,61.4549,17.45,1259.09
59,bytetrade,2022-11-10T18:38:19.472,1268.69,1251.24,18.6384,61.4549,17.45,1260.00


<font color= lightblue> <font size = 2> OHLCV ETH </font>

In [96]:
verif_df2_OH = fn.concatenate(verif_0_E[1], verif_1_E[1], verif_2_E[1], n)
verif_df2_OH.head(10)

,exchange,timestamp,open,high,low,close,volume
0,binance,2022-11-10T18:35:16.488,1260.59,1261.22,1260.55,1260.82,140.8706
1,binance,2022-11-10T18:35:19.352,1260.59,1261.22,1260.55,1260.81,147.2086
2,binance,2022-11-10T18:35:21.798,1260.59,1261.22,1260.55,1260.70,157.7518
3,binance,2022-11-10T18:35:25.257,1260.59,1261.22,1260.55,1260.77,172.9388
4,binance,2022-11-10T18:35:27.654,1260.59,1261.22,1260.55,1260.91,191.3054
5,binance,2022-11-10T18:35:30.056,1260.59,1261.22,1260.55,1260.86,200.4385
6,binance,2022-11-10T18:35:33.453,1260.59,1261.22,1260.55,1260.69,213.9666
7,binance,2022-11-10T18:35:36.452,1260.59,1261.22,1260.55,1260.83,226.3425
8,binance,2022-11-10T18:35:39.016,1260.59,1261.22,1260.55,1260.68,239.6662
9,binance,2022-11-10T18:35:41.404,1260.59,1261.22,1260.40,1260.55,247.9450


In [97]:
verif_df2_OH.tail(10)

,exchange,timestamp,open,high,low,close,volume
50,bytetrade,2022-11-10T18:37:51.755,1259.95,1261.27,1258.84,1259.73,0.0447
51,bytetrade,2022-11-10T18:37:54.640,1259.95,1261.27,1258.84,1259.73,0.0447
52,bytetrade,2022-11-10T18:37:58.846,1259.95,1261.27,1258.84,1259.40,0.0494
53,bytetrade,2022-11-10T18:38:01.451,1260.60,1260.60,1260.60,1260.60,0.0000
54,bytetrade,2022-11-10T18:38:04.970,1260.60,1260.60,1260.60,1260.60,0.0000
55,bytetrade,2022-11-10T18:38:07.708,1260.24,1260.24,1260.24,1260.24,0.0047
56,bytetrade,2022-11-10T18:38:10.964,1260.24,1260.24,1260.24,1260.24,0.0047
57,bytetrade,2022-11-10T18:38:13.930,1260.24,1260.24,1259.13,1259.13,0.0104
58,bytetrade,2022-11-10T18:38:16.717,1260.24,1260.24,1259.09,1259.09,0.0136
59,bytetrade,2022-11-10T18:38:19.472,1260.24,1260.24,1259.09,1260.00,0.0160


<font color= lightblue> <font size = 2> Verification XRP </font>

In [98]:
v_params0= 25, criptos[2], exchanges[0]
v_params1= 25, criptos[2], exchanges[1]
v_params2= 25, criptos[2], exchanges[2]
v_params0, v_params1, v_params2

((25, 'XRP/USDT', 'binance'),
 (25, 'XRP/USDT', 'ftx'),
 (25, 'XRP/USDT', 'bytetrade'))

In [101]:
verif_0_X = vs.verif_ex1(v_params0[0],v_params0[1], v_params0[2], n)
verif_1_X = vs.verif_ex1(v_params1[0],v_params1[1], v_params1[2], n)
verif_2_X = vs.verif_ex1(v_params2[0],v_params2[1], v_params2[2], n)

In this example, bigger spreads can be detected on bytetrade for XRP. On the contrary, binance has lower spreads and bigger volumes as well.

In [102]:
verif_df3 = fn.concatenate(verif_0_X[0], verif_1_X[0], verif_2_X[0], n)
verif_df3.head(10)

,exchange,timestamp,ask,bid,ask_volume,bid_volume,spread,close_price
0,binance,2022-11-10T18:46:15.105,0.3788,0.3787,2276393.0,1632933.0,0.0001,0.3788
1,binance,2022-11-10T18:46:17.687,0.3789,0.3788,2266335.0,1613607.0,0.0001,0.3788
2,binance,2022-11-10T18:46:20.177,0.3789,0.3788,2249896.0,1608963.0,0.0001,0.3788
3,binance,2022-11-10T18:46:22.755,0.3789,0.3788,2289053.0,1622368.0,0.0001,0.3788
4,binance,2022-11-10T18:46:25.516,0.3790,0.3789,2240759.0,1625885.0,0.0001,0.3789
5,binance,2022-11-10T18:46:28.011,0.3790,0.3789,2202227.0,1584276.0,0.0001,0.3790
6,binance,2022-11-10T18:46:30.564,0.3790,0.3789,2281175.0,1681810.0,0.0001,0.3790
7,binance,2022-11-10T18:46:33.204,0.3790,0.3789,2077940.0,1643481.0,0.0001,0.3789
8,binance,2022-11-10T18:46:36.296,0.3792,0.3791,2025475.0,1471723.0,0.0001,0.3792
9,binance,2022-11-10T18:46:39.125,0.3793,0.3792,2050374.0,1484022.0,0.0001,0.3792


In [103]:
verif_df3.tail(10)

,exchange,timestamp,ask,bid,ask_volume,bid_volume,spread,close_price
50,bytetrade,2022-11-10T18:48:40.582,0.397543,0.357739,7.434,2495.6248,0.039804,0.377465
51,bytetrade,2022-11-10T18:48:43.308,0.397543,0.357739,7.434,2250.3335,0.039804,0.377732
52,bytetrade,2022-11-10T18:48:45.752,0.397543,0.357739,7.434,2250.3335,0.039804,0.377732
53,bytetrade,2022-11-10T18:48:48.677,0.397543,0.357739,7.434,2417.1554,0.039804,0.377732
54,bytetrade,2022-11-10T18:48:51.801,0.397543,0.357739,7.434,2417.1554,0.039804,0.377990
55,bytetrade,2022-11-10T18:48:54.450,0.397543,0.357739,7.434,2373.8037,0.039804,0.377971
56,bytetrade,2022-11-10T18:48:57.144,0.397543,0.357739,7.434,2373.8037,0.039804,0.377271
57,bytetrade,2022-11-10T18:48:59.544,0.397543,0.357739,7.434,2373.8037,0.039804,0.377271
58,bytetrade,2022-11-10T18:49:03.211,0.397543,0.357739,7.434,2373.8037,0.039804,0.377590
59,bytetrade,2022-11-10T18:49:05.701,0.397543,0.357739,7.434,2373.8037,0.039804,0.377692


<font color= lightblue> <font size = 2> OHLCV XRP </font>

AS it can be seen volume is almost null in certain timestamps for some exchanges and this represent liquidity issues so lower prices shouldn´t be always better but depending on the situation.

In [134]:
verif_df3_OH = fn.concatenate(verif_0_X[1], verif_1_X[1], verif_2_X[1], n)
verif_df3_OH.head(30)

,exchange,timestamp,open,high,low,close,volume
0,binance,2022-11-10T18:46:15.105,0.3789,0.37900,0.3787,0.37880,31910.0000
1,binance,2022-11-10T18:46:17.687,0.3789,0.37900,0.3787,0.37880,33876.0000
2,binance,2022-11-10T18:46:20.177,0.3789,0.37900,0.3787,0.37880,33991.0000
3,binance,2022-11-10T18:46:22.755,0.3789,0.37900,0.3787,0.37880,35886.0000
4,binance,2022-11-10T18:46:25.516,0.3789,0.37900,0.3787,0.37890,39507.0000
5,binance,2022-11-10T18:46:28.011,0.3789,0.37900,0.3787,0.37900,40009.0000
6,binance,2022-11-10T18:46:30.564,0.3789,0.37900,0.3787,0.37900,40273.0000
7,binance,2022-11-10T18:46:33.204,0.3789,0.37900,0.3787,0.37890,48323.0000
8,binance,2022-11-10T18:46:36.296,0.3789,0.37920,0.3787,0.37920,84339.0000
9,binance,2022-11-10T18:46:39.125,0.3789,0.37920,0.3787,0.37920,84339.0000


In [105]:
verif_df3_OH.tail(10)

,exchange,timestamp,open,high,low,close,volume
50,bytetrade,2022-11-10T18:48:40.582,0.377454,0.377819,0.377444,0.377465,0.2520
51,bytetrade,2022-11-10T18:48:43.308,0.377454,0.377819,0.377444,0.377732,0.2622
52,bytetrade,2022-11-10T18:48:45.752,0.377454,0.377819,0.377444,0.377732,0.2622
53,bytetrade,2022-11-10T18:48:48.677,0.377454,0.377819,0.377444,0.377732,0.2622
54,bytetrade,2022-11-10T18:48:51.801,0.377454,0.377990,0.377444,0.377990,0.2706
55,bytetrade,2022-11-10T18:48:54.450,0.377454,0.377990,0.377444,0.377971,0.2905
56,bytetrade,2022-11-10T18:48:57.144,0.377454,0.377990,0.377271,0.377271,0.3172
57,bytetrade,2022-11-10T18:48:59.544,0.377454,0.377990,0.377271,0.377271,0.3172
58,bytetrade,2022-11-10T18:49:03.211,0.377590,0.377590,0.377590,0.377590,0.0300
59,bytetrade,2022-11-10T18:49:05.701,0.377590,0.377692,0.377590,0.377692,0.0414


<font color= orange> <font size = 5> 2. Microstructure Visualization </font>

Now we are going to obtain total_volumes, mid_prices and VWAP metric for criptocurrencies in every exchange.

In [106]:
help(vs.Micro)

Help on function Micro in module visualizations:

Micro(lvls, cripto, exchange, n, ts)
    Function that returns Microstructure OB and OHLCV for i levels (as integer) n times 
    from specified criptocurrency ('BTC/USDT','ETH/USDT','XRP/USDT', or others) and the 
    following exchanges: binance, ftx or bytetrade (as string).
    
    Parameters:
    ----------
    lvls: Levels of bids/asks in the Order Book (int).
    cripto: Criptocurrency downloadable symbol (str).
    exchanges: Criptocurrency downloadable Exchange (str).
    n: Data retrieval per exchange (int).
    ts: timesleep (s) required in between data retrieval.
    
    Returns:
    -------
    Micro[0]: Order Books Data.
    Micro[1]: OHLCV Data.



<strong> 
<font color= lightgreen> <font size = 3> Progress check 2  </font> <strong>

Dataframes

<font color= lightblue> <font size = 2> Verification BTC </font>

In [107]:
params0= 25, criptos[0], exchanges[0]
params1= 25, criptos[0], exchanges[1]
params2= 25, criptos[0], exchanges[2]
params0,params1,params2

((25, 'BTC/USDT', 'binance'),
 (25, 'BTC/USDT', 'ftx'),
 (25, 'BTC/USDT', 'bytetrade'))

In [108]:
MS_0_B = vs.Micro(params0[0],params0[1], params0[2], n, .01)
MS_1_B = vs.Micro(params1[0],params1[1], params1[2], n, .01)
MS_2_B = vs.Micro(params2[0],params2[1], params2[2], n, .01)

Bid and Asks Volumes in the Microstructure determine the demand and it's a factor for price movements. This is also true for the VWAP which is a trend detector by reflecting the price adjusted by its volume.

In [109]:
MS_B = fn.concatenate(MS_0_B[0], MS_1_B[0], MS_2_B[0], n)
MS_B.head(10)

,exchange,timestamp,level,ask_volume,bid_volume,total_volume,mid_price,vwap
0,binance,2022-11-10T18:49:09.439,25,1.73814,2.27489,4.01303,17362.385,21688.230707
1,binance,2022-11-10T18:49:10.901,25,1.89162,1.98899,3.88061,17362.230,21835.815951
2,binance,2022-11-10T18:49:12.934,25,1.76857,1.31908,3.08765,17361.630,22984.022679
3,binance,2022-11-10T18:49:15.033,25,2.24064,2.79038,5.03102,17359.645,20809.602092
4,binance,2022-11-10T18:49:16.465,25,0.54957,0.98673,1.53630,17357.925,28656.249954
5,binance,2022-11-10T18:49:18.090,25,3.60847,0.77118,4.37965,17349.465,21310.464640
6,binance,2022-11-10T18:49:19.480,25,1.92561,11.49867,13.42428,17346.790,18638.402117
7,binance,2022-11-10T18:49:20.905,25,0.92951,0.80107,1.73058,17343.520,27364.910888
8,binance,2022-11-10T18:49:22.370,25,0.63150,1.23940,1.87090,17346.875,26618.646405
9,binance,2022-11-10T18:49:23.917,25,4.05767,0.54448,4.60215,17342.710,21110.774295


As it will be demonstrated in *progress check (visualizations)* mid_prices vary significantly between exchanges.

In [137]:
MS_B.tail(10)

,exchange,timestamp,level,ask_volume,bid_volume,total_volume,mid_price,vwap
timestamp,,,,,,,,
2022-11-10 18:50:41.353,bytetrade,2022-11-10T18:50:41.353,25,3.1563,2.844773,6.001073,17357.9,20138.445395
2022-11-10 18:50:42.832,bytetrade,2022-11-10T18:50:42.832,25,3.1563,2.844773,6.001073,17357.9,20138.445395
2022-11-10 18:50:45.402,bytetrade,2022-11-10T18:50:45.402,25,3.1563,2.844773,6.001073,17357.9,20138.445395
2022-11-10 18:50:47.123,bytetrade,2022-11-10T18:50:47.123,25,3.1563,2.844773,6.001073,17357.9,20138.445395
2022-11-10 18:50:48.552,bytetrade,2022-11-10T18:50:48.552,25,3.1563,2.844773,6.001073,17357.9,20138.445395
2022-11-10 18:50:50.173,bytetrade,2022-11-10T18:50:50.173,25,3.1563,2.844773,6.001073,17357.9,20138.445395
2022-11-10 18:50:51.625,bytetrade,2022-11-10T18:50:51.625,25,3.1563,2.844773,6.001073,17357.9,20138.445395
2022-11-10 18:50:53.066,bytetrade,2022-11-10T18:50:53.066,25,3.1563,2.844773,6.001073,17357.9,20138.445395
2022-11-10 18:50:54.736,bytetrade,2022-11-10T18:50:54.736,25,3.1563,2.844773,6.001073,17357.9,20138.445395


<font color= lightblue> <font size = 2> Verification ETH </font>

In [111]:
params0= 25, criptos[1], exchanges[0]
params1= 25, criptos[1], exchanges[1]
params2= 25, criptos[1], exchanges[2]
params0,params1,params2

((25, 'ETH/USDT', 'binance'),
 (25, 'ETH/USDT', 'ftx'),
 (25, 'ETH/USDT', 'bytetrade'))

In [113]:
MS_0_E = vs.Micro(params0[0],params0[1], params0[2], n, .01)
MS_1_E = vs.Micro(params1[0],params1[1], params1[2], n, .01)
MS_2_E = vs.Micro(params2[0],params2[1], params2[2], n, .01)

mid_prices differences can be attributed to liquidity for this sample. Volumes shouldn't be taken lightly, that's why the vwap is an important metric to measure and estimate prices. 

In [139]:
MS_E = fn.concatenate(MS_0_E[0], MS_1_E[0], MS_2_E[0], n)
MS_E.head(10)

,exchange,timestamp,level,ask_volume,bid_volume,total_volume,mid_price,vwap
0,binance,2022-11-10T18:53:24.596,25,42.7740,64.0866,106.8606,1267.975,1279.835739
1,binance,2022-11-10T18:53:25.977,25,33.0333,67.4832,100.5165,1267.975,1280.584645
2,binance,2022-11-10T18:53:27.353,25,46.6031,58.2039,104.8070,1267.995,1280.088429
3,binance,2022-11-10T18:53:29.384,25,33.4461,63.9716,97.4177,1268.075,1281.086936
4,binance,2022-11-10T18:53:31.437,25,29.0975,81.9624,111.0599,1268.165,1279.578793
5,binance,2022-11-10T18:53:32.954,25,52.9999,564.5006,617.5005,1268.185,1270.233747
6,binance,2022-11-10T18:53:34.810,25,42.7205,68.3068,111.0273,1268.155,1279.572056
7,binance,2022-11-10T18:53:36.743,25,37.9953,47.0161,85.0114,1268.155,1283.067529
8,binance,2022-11-10T18:53:38.282,25,92.6565,555.1344,647.7909,1268.075,1270.027545
9,binance,2022-11-10T18:53:39.726,25,50.4381,561.1970,611.6351,1267.885,1269.952952


In [115]:
MS_E.tail(10)

,exchange,timestamp,level,ask_volume,bid_volume,total_volume,mid_price,vwap
50,bytetrade,2022-11-10T18:55:04.227,25,18.0722,61.3559,79.4281,1268.205,1275.181336
51,bytetrade,2022-11-10T18:55:05.585,25,18.0722,61.3559,79.4281,1268.205,1275.181336
52,bytetrade,2022-11-10T18:55:07.218,25,18.0722,61.3559,79.4281,1268.205,1275.181336
53,bytetrade,2022-11-10T18:55:08.834,25,18.0722,61.3559,79.4281,1268.205,1275.181336
54,bytetrade,2022-11-10T18:55:10.233,25,18.0722,61.3559,79.4281,1268.205,1275.181336
55,bytetrade,2022-11-10T18:55:12.878,25,18.0722,61.3559,79.4281,1268.205,1275.181336
56,bytetrade,2022-11-10T18:55:14.458,25,18.0722,61.3559,79.4281,1268.205,1275.181336
57,bytetrade,2022-11-10T18:55:15.896,25,18.0722,61.3559,79.4281,1268.205,1275.181336
58,bytetrade,2022-11-10T18:55:18.099,25,18.0722,61.3559,79.4281,1268.205,1275.181336
59,bytetrade,2022-11-10T18:55:19.830,25,18.0722,61.3559,79.4281,1268.205,1275.181336


<font color= lightblue> <font size = 2> Verification XRP </font>

In [116]:
params0= 25, criptos[2], exchanges[0]
params1= 25, criptos[2], exchanges[1]
params2= 25, criptos[2], exchanges[2]
params0,params1,params2

((25, 'XRP/USDT', 'binance'),
 (25, 'XRP/USDT', 'ftx'),
 (25, 'XRP/USDT', 'bytetrade'))

In [117]:
MS_0_X = vs.Micro(params0[0],params0[1], params0[2], n, .01)
MS_1_X = vs.Micro(params1[0],params1[1], params1[2], n, .01)
MS_2_X = vs.Micro(params2[0],params2[1], params2[2], n, .01)

Bid or Asks Volumes can actually be non-existent for certain assets in some exchanges but this may not always reduce the vwap significantly.

In [145]:
MS_X = fn.concatenate(MS_0_X[0], MS_1_X[0], MS_2_X[0], n)
MS_X.head(10)

,exchange,timestamp,level,ask_volume,bid_volume,total_volume,mid_price,vwap
0,binance,2022-11-10T18:55:22.240,25,1427432.0,1268289.0,2695721.0,0.37905,0.3790
1,binance,2022-11-10T18:55:23.695,25,1524708.0,1314789.0,2839497.0,0.37915,0.3791
2,binance,2022-11-10T18:55:25.096,25,1513576.0,1312371.0,2825947.0,0.37915,0.3791
3,binance,2022-11-10T18:55:26.549,25,1515657.0,1273988.0,2789645.0,0.37915,0.3791
4,binance,2022-11-10T18:55:27.944,25,1512580.0,1279561.0,2792141.0,0.37915,0.3791
5,binance,2022-11-10T18:55:29.365,25,970471.0,1298325.0,2268796.0,0.37955,0.3795
6,binance,2022-11-10T18:55:30.796,25,995037.0,1296898.0,2291935.0,0.37955,0.3795
7,binance,2022-11-10T18:55:32.799,25,960952.0,1232979.0,2193931.0,0.37955,0.3795
8,binance,2022-11-10T18:55:35.673,25,919844.0,1175991.0,2095835.0,0.37975,0.3797
9,binance,2022-11-10T18:55:37.386,25,953798.0,1172314.0,2126112.0,0.37975,0.3797


In [119]:
MS_X.tail(10)

,exchange,timestamp,level,ask_volume,bid_volume,total_volume,mid_price,vwap
50,bytetrade,2022-11-10T18:56:57.184,25,7.434,2289.945,2297.379,0.377641,0.357912
51,bytetrade,2022-11-10T18:56:58.821,25,7.434,2289.945,2297.379,0.377641,0.357912
52,bytetrade,2022-11-10T18:57:00.511,25,7.434,2289.945,2297.379,0.377641,0.357912
53,bytetrade,2022-11-10T18:57:01.993,25,7.434,2289.945,2297.379,0.377641,0.357912
54,bytetrade,2022-11-10T18:57:03.729,25,7.434,2289.945,2297.379,0.377641,0.357912
55,bytetrade,2022-11-10T18:57:05.336,25,7.434,2289.945,2297.379,0.377641,0.357912
56,bytetrade,2022-11-10T18:57:06.754,25,7.434,2289.945,2297.379,0.377641,0.357912
57,bytetrade,2022-11-10T18:57:09.345,25,7.434,2289.945,2297.379,0.377641,0.357912
58,bytetrade,2022-11-10T18:57:10.771,25,7.434,2289.945,2297.379,0.377641,0.357912
59,bytetrade,2022-11-10T18:57:12.439,25,7.434,2289.945,2297.379,0.377641,0.357912


<strong> 
<font color= lightgreen> <font size = 3> Progress check 2  </font> <strong>

Visualization

In [120]:
help(vs.Plot_line)

Help on function Plot_line in module visualizations:

Plot_line(data_ms, cripto, title)
    Function that plots Mid-Prices (y) and timestamps (x) for exchanges in data.
    Parameters:
    ----------
    data_ms: Microstructure data for column extraction ['exchanges', 'Mid_Prices'] and index timestamp as datetime.
    cripto: Downloadable cripto symbol.
    title: Title of facet_cols plotly lines before criptocurrency symbol (ex: 'Volume of': XRP/USDT)
    
    Returns:
    -------
    facet_col plots for Mid-Prices (col) during timestamps of data (index) for exchanges (col).



As it can be appreciated, Mid-Prices have different values in every exchange. Given this fact, there are entities that aim to reduce the spread between exchanges and they intervene creating virtual demand but let's not ignore that this is also traduced into gains for them, and they are also known as market makers.

In [121]:
vs.Plot_line(MS_B, criptos[0], "Mid-Price of")

Within exchanges with liquidity issues, volatility tends to be higher as well and this fact increases risks even further.

In [122]:
vs.Plot_line(MS_E, criptos[1], "Mid-Price of")

There are exchanges that have bigger or smaller price differences but they all have them which means that market makers will always be there to reduce them and take this small proportion as their gains but for big participations.

In [123]:
vs.Plot_line(MS_X, criptos[2], "Mid-Price of")

<font color= orange> <font size = 5> 3. Microstructure Modelling: </font>

<strong> 
<font color= lightgreen> <font size = 3> Progress check 3  </font> <strong>

<font color= lightblue> <font size = 2> BTC Spreads </font>

The effective spread lags number is defined:

In [124]:
lags=5

As it can be appreciated, spreads and Roll's effective spreads (5 lags) values vary significantly.

In [125]:
Roll_B = vs.roll_model(verif_df1, lags).reset_index(drop=True)
Roll_B.head(10)

,exchange,timestamp,close_price,spread,effective_spread
0,binance,2022-11-10T18:32:35.140,17272.10,2.04,3.154378
1,binance,2022-11-10T18:32:37.636,17279.84,1.28,2.706012
2,binance,2022-11-10T18:32:40.288,17278.12,1.16,4.626536
3,binance,2022-11-10T18:32:42.753,17279.90,0.59,3.728793
4,binance,2022-11-10T18:32:45.116,17279.70,1.62,2.570512
5,binance,2022-11-10T18:32:47.777,17281.12,1.87,3.183541
6,binance,2022-11-10T18:32:50.795,17276.59,1.67,2.311825
7,binance,2022-11-10T18:32:53.215,17276.16,1.60,3.916916
8,binance,2022-11-10T18:32:55.584,17267.08,1.04,2.475601
9,binance,2022-11-10T18:32:57.999,17272.40,0.57,4.834425


In [126]:
Roll_B.tail(10)

,exchange,timestamp,close_price,spread,effective_spread
45,bytetrade,2022-11-10T18:34:48.048,17272.7,256.7,4.952104
46,bytetrade,2022-11-10T18:34:50.729,17264.8,256.7,9.797023
47,bytetrade,2022-11-10T18:34:53.407,17271.3,256.7,7.886634
48,bytetrade,2022-11-10T18:34:56.918,17255.0,256.7,6.059785
49,bytetrade,2022-11-10T18:34:59.461,17258.4,256.7,8.592865
50,bytetrade,2022-11-10T18:35:02.243,17261.4,256.7,6.401224
51,bytetrade,2022-11-10T18:35:04.938,17263.9,256.7,1.554992
52,bytetrade,2022-11-10T18:35:07.738,17258.2,256.7,4.000750
53,bytetrade,2022-11-10T18:35:10.427,17269.3,256.7,4.658040
54,bytetrade,2022-11-10T18:35:12.769,17264.7,256.7,5.861143


Some exchanges have bigger spreads but Roll's effective Spread shouldn't be as high, specially if lags no° is increased and the data contains several exchanges.

In [127]:
vs.Roll_plot(Roll_B, criptos[0], 'Spreads of')

<font color= lightblue> <font size = 2> ETH Spreads </font>

In [128]:
Roll_E = vs.roll_model(verif_df2, lags).reset_index(drop=True)
Roll_E.head(10)

,exchange,timestamp,close_price,spread,effective_spread
0,binance,2022-11-10T18:35:30.056,1260.86,0.01,0.081035
1,binance,2022-11-10T18:35:33.453,1260.69,0.01,0.226495
2,binance,2022-11-10T18:35:36.452,1260.83,0.01,0.070475
3,binance,2022-11-10T18:35:39.016,1260.68,0.01,0.259358
4,binance,2022-11-10T18:35:41.404,1260.55,0.01,0.177294
5,binance,2022-11-10T18:35:43.785,1260.46,0.01,0.154380
6,binance,2022-11-10T18:35:46.602,1260.59,0.01,0.221284
7,binance,2022-11-10T18:35:49.012,1260.66,0.01,0.107703
8,binance,2022-11-10T18:35:51.949,1260.71,0.01,0.174929
9,binance,2022-11-10T18:35:55.773,1261.51,0.01,0.180000


In [129]:
Roll_E.tail(10)

,exchange,timestamp,close_price,spread,effective_spread
45,bytetrade,2022-11-10T18:37:51.755,1259.73,17.45,0.361248
46,bytetrade,2022-11-10T18:37:54.640,1259.73,17.45,0.580747
47,bytetrade,2022-11-10T18:37:58.846,1259.40,17.45,0.022361
48,bytetrade,2022-11-10T18:38:01.451,1260.60,17.45,0.647405
49,bytetrade,2022-11-10T18:38:04.970,1260.60,17.45,0.549060
50,bytetrade,2022-11-10T18:38:07.708,1260.24,17.45,0.422729
51,bytetrade,2022-11-10T18:38:10.964,1260.24,17.45,0.419404
52,bytetrade,2022-11-10T18:38:13.930,1259.13,17.45,0.507149
53,bytetrade,2022-11-10T18:38:16.717,1259.09,17.45,0.254755
54,bytetrade,2022-11-10T18:38:19.472,1260.00,17.45,0.355762


The spreads for exchanges with liquidity issues tend to have bigger movements and volatility. This is reduced by the Roll's Effective Spread with n lags, which means that it's a valuable metric to use when the data is avaliable and the n number of exchanges are greater than just a few because liquidity issues arise inevitably.

In [130]:
vs.Roll_plot(Roll_E, criptos[1], 'Spreads of')

<font color= lightblue> <font size = 2> XRP Spreads </font>

In [131]:
Roll_X = vs.roll_model(verif_df3, lags).reset_index(drop=True)
Roll_X.head(10)

,exchange,timestamp,close_price,spread,effective_spread
0,binance,2022-11-10T18:46:28.011,0.3790,0.0001,0.000082
1,binance,2022-11-10T18:46:30.564,0.3790,0.0001,0.000115
2,binance,2022-11-10T18:46:33.204,0.3789,0.0001,0.000000
3,binance,2022-11-10T18:46:36.296,0.3792,0.0001,0.000141
4,binance,2022-11-10T18:46:39.125,0.3792,0.0001,0.000082
5,binance,2022-11-10T18:46:41.552,0.3792,0.0001,0.000163
6,binance,2022-11-10T18:46:44.021,0.3793,0.0001,0.000115
7,binance,2022-11-10T18:46:46.853,0.3793,0.0001,0.000115
8,binance,2022-11-10T18:46:49.294,0.3793,0.0001,0.000082
9,binance,2022-11-10T18:46:52.011,0.3792,0.0001,0.000082


In [132]:
Roll_X.tail(10)

,exchange,timestamp,close_price,spread,effective_spread
45,bytetrade,2022-11-10T18:48:40.582,0.377465,0.039804,0.000128
46,bytetrade,2022-11-10T18:48:43.308,0.377732,0.039804,0.000091
47,bytetrade,2022-11-10T18:48:45.752,0.377732,0.039804,0.000139
48,bytetrade,2022-11-10T18:48:48.677,0.377732,0.039804,0.000128
49,bytetrade,2022-11-10T18:48:51.801,0.377990,0.039804,0.000139
50,bytetrade,2022-11-10T18:48:54.450,0.377971,0.039804,0.000024
51,bytetrade,2022-11-10T18:48:57.144,0.377271,0.039804,0.000126
52,bytetrade,2022-11-10T18:48:59.544,0.377271,0.039804,0.000255
53,bytetrade,2022-11-10T18:49:03.211,0.377590,0.039804,0.000250
54,bytetrade,2022-11-10T18:49:05.701,0.377692,0.039804,0.000263


In some cases, Roll's effective spread values don't vary significatly from actual spreads and this would be a sign of a healthy market but only for average traders.

In [133]:
vs.Roll_plot(Roll_X, criptos[2], 'Spreads of')

### <font color= orange> <font size = 2> **Note:** *Conclusions contained in dataframes and plot descriptions* </font>



### <font color= orange> <font size = 5> 5. Bibliography: </font>

<font color= lightgray>

+ Villalobos, O, 2021. **Python Project Template**. Available at: https://github.com/OmarVillalobos/python-project. 
(Accessed: November 5, 2022). 

+ Financial Engineering, **Marketmicrostructure.pptx**. ITESO -  Microstructure and Trading Systems (ITE1731 - DMAF). 
Available at: https://docs.google.com/presentation/d/1uYyMERKKGeZ0MPqbJ3bcir9lfpzbeAH9ZMmxbYysokQ/edit?usp=sharing (Accessed: November 5, 2022). 

+ Muñoz E., J **Roll Model** *(spanish)*. ITESO - (ITE1731 - DMAF) Available at: https://github.com/EstebanMqz/MyST_LAB_4/blob/main/Roll%20.pdf (Accessed: November 5, 2022).

+ Muñoz E., J **Asset Pricing Theory** *(spanish)*. ITESO - (ITE1731 - DMAF) Available at: https://github.com/EstebanMqz/MyST_LAB_4/blob/main/APT.pdf (Accessed: November 5, 2022). 